# Getting Started with MLB Player Digital Engagement Forecasting #


This is *MLB Digital Engagement Forecasting* competition held by Kaggle. This notebook will:
- load and join the data,
- create a feature set,
- predict with trained model.
- make a submission.




In [1]:
import gc
import sys
import warnings
from joblib import Parallel, delayed
from pathlib import Path
import datetime
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from statsmodels.tsa.deterministic import (CalendarFourier,
                                           CalendarSeasonality,
                                           CalendarTimeTrend,
                                           DeterministicProcess)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers.experimental.preprocessing import StringLookup

warnings.simplefilter("ignore")

# Create Training Data #

### Read and extract dataframes

In [2]:
# Helper function to unpack json found in daily data
def unpack_json(json_str):
    return pd.DataFrame() if pd.isna(json_str) else pd.read_json(json_str)


def unpack_data(data, dfs=None, n_jobs=-1):
    if dfs is not None:
        data = data.loc[:, dfs]
    unnested_dfs = {}
    for name, column in data.iteritems():
        daily_dfs = Parallel(n_jobs=n_jobs)(
            delayed(unpack_json)(item) for date, item in column.iteritems())
        df = pd.concat(daily_dfs)
        unnested_dfs[name] = df
    return unnested_dfs

We use features from following tables: seasons,teams,players,and awards

In [3]:
data_dir = Path('../input/mlb-player-digital-engagement-forecasting/')

df_names = ['seasons', 'teams', 'players', 'awards']

for name in df_names:
    globals()[name] = pd.read_csv(data_dir / f"{name}.csv")

kaggle_data_tabs = widgets.Tab()
# Add Output widgets for each pandas DF as tabs' children
kaggle_data_tabs.children = list([widgets.Output() for df_name in df_names])

for index in range(0, len(df_names)):
    # Rename tab bar titles to df names
    kaggle_data_tabs.set_title(index, df_names[index])
    
    # Display corresponding table output for this tab name
    with kaggle_data_tabs.children[index]:
        display(eval(df_names[index]))

display(kaggle_data_tabs)
del teams

**Players tables(- Library containing high level information about all MLB players in this dataset)** :Creating dummies for players birthcountry. Features used in the model:birthyear, birthcountry, and primary position code

In [4]:
players_1=players[players['playerForTestSetAndFuturePreds']==True].copy()
players_1.loc[players_1['playerForTestSetAndFuturePreds']==True,"birthCountry"].unique()
players_1.loc[(players_1['birthCountry']=='Taiwan')|(players_1['birthCountry']=='South Korea')|(players_1['birthCountry']=='Japan')|(players_1['birthCountry']=="China"),"birthCountry"]="Asia"
players_1.loc[(players_1['birthCountry']=='Germany')|(players_1['birthCountry']=='Netherlands'),'birthCountry']='Europe'
players_1.loc[(players_1['birthCountry']!='USA')&(players_1['birthCountry']!='Europe')&(players_1['birthCountry']!='Asia'),'birthCountry']='America'
players_1=players_1[['playerId','birthCountry','DOB','heightInches','weight','primaryPositionCode']]
players_1=pd.get_dummies(players_1,columns=['birthCountry','primaryPositionCode'])
players_1['DOB'] = pd.PeriodIndex(players_1.DOB, freq='D')
players_1['birthyear']=players_1.DOB.dt.year
players_1.drop('DOB',axis=1,inplace=True)

The training data is a time-indexed collection of nested JSON fields containing information about each player. Our target is the `nextDayPlayerEngagement` column, while the remaining columns could be used to construct features.

In [5]:
%%time
# Define dataframes to load from training set
dfs = [
    'nextDayPlayerEngagement',  # targets
    'playerBoxScores',  # features
    # Other dataframes available for features:
    'games',
    'rosters',
    #'teamBoxScores',
    'transactions',
    'standings',
    'awards',
    # 'events',
    'playerTwitterFollowers',
]

# Read training data
training = pd.read_csv(
    data_dir / 'train.csv',
    usecols=['date'] + dfs,
)

# Convert training data date field to datetime type
training['date'] = pd.to_datetime(training['date'], format="%Y%m%d")
training = training.set_index('date').to_period('D')
print(training.info())

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 1216 entries, 2018-01-01 to 2021-04-30
Freq: D
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   nextDayPlayerEngagement  1216 non-null   object
 1   games                    639 non-null    object
 2   rosters                  1216 non-null   object
 3   playerBoxScores          538 non-null    object
 4   transactions             1103 non-null   object
 5   standings                531 non-null    object
 6   awards                   294 non-null    object
 7   playerTwitterFollowers   40 non-null     object
dtypes: object(8)
memory usage: 85.5+ KB
None
CPU times: user 32.2 s, sys: 9.47 s, total: 41.6 s
Wall time: 1min 10s


In [6]:
training.head()

,nextDayPlayerEngagement,games,rosters,playerBoxScores,transactions,standings,awards,playerTwitterFollowers
date,,,,,,,,
2018-01-01,"[{""engagementMetricsDate"":""2018-01-02"",""player...",NaN,"[{""playerId"":400121,""gameDate"":""2018-01-01"",""t...",NaN,"[{""transactionId"":340732,""playerId"":547348,""pl...",NaN,NaN,"[{""date"":""2018-01-01"",""playerId"":545361,""playe..."
2018-01-02,"[{""engagementMetricsDate"":""2018-01-03"",""player...",NaN,"[{""playerId"":134181,""gameDate"":""2018-01-02"",""t...",NaN,"[{""transactionId"":339458,""playerId"":621173,""pl...",NaN,NaN,NaN
2018-01-03,"[{""engagementMetricsDate"":""2018-01-04"",""player...",NaN,"[{""playerId"":425492,""gameDate"":""2018-01-03"",""t...",NaN,"[{""transactionId"":347527,""playerId"":572389,""pl...",NaN,NaN,NaN
2018-01-04,"[{""engagementMetricsDate"":""2018-01-05"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-04"",""t...",NaN,"[{""transactionId"":339549,""playerId"":545343,""pl...",NaN,NaN,NaN
2018-01-05,"[{""engagementMetricsDate"":""2018-01-06"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-05"",""t...",NaN,"[{""transactionId"":341195,""playerId"":628336,""pl...",NaN,NaN,NaN


In [7]:
%time
# Unpack nested dataframes and store in dictionary `training_dfs`
training_dfs = unpack_data(training, dfs=dfs)
print('\n', training_dfs.keys())

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs

 dict_keys(['nextDayPlayerEngagement', 'playerBoxScores', 'games', 'rosters', 'transactions', 'standings', 'awards', 'playerTwitterFollowers'])


**Games Table:**
Transfer game per row data structure to team per row structure.Features newly created for the model:wins difference,score difference 


In [8]:
def make_games(dfs:dict): 
    home=dfs['games'].copy()
    if "gameDate" in  home.columns:
        home=home.rename(columns={'homeId':'teamId','homeWins':'teamWins','awayWins':'oppWins','homeScore':'teamScore','awayScore':'oppScore','homeWinner':'teamWinner',"gameDate":"date"})
        home=home[['teamId','teamWins','oppWins','teamScore','oppScore','teamWinner',"date","gameType"]]
        away=dfs['games'].copy()
        away=away.rename(columns={'awayId':'teamId','homeWins':'oppWins','awayWins':'teamWins','homeScore':'oppScore','awayScore':'teamScore','awayWinner':'teamWinner',"gameDate":"date"})
        away=away[['teamId','teamWins','oppWins','teamScore','oppScore','teamWinner',"date","gameType"]]
        away
        games=pd.concat([home,away],axis=0)
        games["winsdiff"]=games['teamWins']-games['oppWins']
        games["scorediff"]=games['teamScore']-games['oppScore']
        games=games.drop(['teamWins','oppWins','teamScore','oppScore'],axis=1)
        games = games.groupby(['date', 'teamId'], as_index=False).mean()
        games['date']=pd.PeriodIndex(games.date,freq='D')
    else:
        games=dfs['games'].copy()
        games[['teamId']]=np.nan
        games[['date']]=np.nan
    return games

#  **Include players' average target value of the previous year as features**

In [9]:
targ = training_dfs['nextDayPlayerEngagement'].copy()
targets = ["target1", "target2", "target3", "target4"]
targ  = targ .astype({name: np.float32 for name  in targets})
targ = targ.rename(columns={'engagementMetricsDate': 'date'})
targ['date'] = pd.to_datetime(targ['date'])
targ = targ.set_index('date').to_period('D')
targ.index = targ.index - 1
targ=targ.reset_index()

In [10]:
targ['seasonId']=targ.date.dt.year+1
targ.drop(['date'],axis=1,inplace=True)
targ=targ.groupby(by=['seasonId','playerId'],as_index=False).mean()
targ=targ.loc[(targ.seasonId!=2022),:]
# Match target dates to feature dates and create date index
targ=targ.rename(columns={"target1":"meantarget1","target2":"meantarget2","target3":"meantarget3","target4":"meantarget4"})
targ = targ.astype({'playerId': str})

# Rosters: player_id, team_id, status_code,game_date

In [11]:
def make_rosters(dfs:dict):
    rosters=dfs['rosters'].copy()
    if "playerId" in rosters.columns:
        rosters=rosters[['playerId','gameDate','teamId','statusCode']]
        rosters=pd.get_dummies(rosters,columns=['statusCode'])
        rosters=rosters.rename(columns={'gameDate':'date'})
        rosters['date']=pd.PeriodIndex(rosters.date,freq='D')
    else:
        rosters[['playerId']]=np.nan
        rosters[['date']]=np.nan
        rosters[['teamId']]=np.nan
    return rosters

# twitter followers: player_id, date, numberof followers

In [12]:

def make_playerTwitterFollowers(dfs:dict):
    twitfollowers=dfs['playerTwitterFollowers'].copy()
    if 'playerId' in  twitfollowers.columns:
        twitfollowers=twitfollowers[['date','playerId','numberOfFollowers']]
        twitfollowers['date']=pd.PeriodIndex(twitfollowers.date,freq='D')
    else:
        twitfollowers[['playerId']]=np.nan
        twitfollowers[['date']]=np.nan
    return twitfollowers


# Team_Standing:'teamid','gamedate','divisionRank','leagueRank','wildCardRank','pct','wins','eliminated','wildCardEliminated'

In [13]:
standings_feature=['divisionRank','leagueRank','wildCardRank','pct','wins','eliminationNumber','wildCardEliminationNumber'
       ]
def make_standings(dfs:dict,standings_feature):
    standings=dfs['standings'].copy()
    if 'teamId' in standings.columns: 
        standings=standings[['teamId','gameDate']+standings_feature]
        standings=standings.rename(columns={'gameDate':'date'})
        standings['date']=pd.PeriodIndex(standings.date,freq='D')
        standings['eliminated']=standings['eliminationNumber']=='E'
        standings['wildeliminated']=standings['wildCardEliminationNumber']=='E'
        standings=standings.drop(['eliminationNumber','wildCardEliminationNumber'],axis=1)
    else:
        standings[['teamId']]=np.nan
        standings[['date']]=np.nan
    return standings

# Previous year award

In [14]:
awardhist=training_dfs['awards'].copy()

In [15]:
awards=['All-MLB First Team', 'All-MLB Second Team','NL Comeback Player of the Year',
       'Edgar Martinez Outstanding DH of the Year',
       'AL Comeback Player of the Year', 'The Hutch Award','AL Cy Young', 'NL Cy Young', 'AL MVP',
       'NL MVP','MLB Players Choice NL Comeback Player',
       'MLB Players Choice NL Outstanding Pitcher',
       'MLB Players Choice NL Outstanding Rookie',
       'MLB Players Choice NL Outstanding Player',
       'Rawlings NL Platinum Glove', 'Babe Ruth Award',
       'MLB Players Choice Player of the Year',
       'MLB Players Choice Man of the Year',
       'MLB Players Choice AL Outstanding Player',
       'MLB Players Choice AL Comeback Player',
       'MLB Players Choice AL Outstanding Pitcher',
       'MLB Players Choice AL Outstanding Rookie', 'Wilson Defensive Player of the Year',
       'Wilson MLB Defensive Player of the Year', 'AL Silver Slugger',
       'NL Silver Slugger', 'MLBPAA Heart and Hustle Award',
       'Rawlings AL Platinum Glove', 'Roberto Clemente Award', 'NL Hank Aaron Award',
       'AL Hank Aaron Award', 'Trevor Hoffman NL Reliever of the Year',
       'Mariano Rivera AL Reliever of the Year',
       'World Series Championship', 'World Series MVP','Baseball America Minor League Player of the Year',
       'Baseball America Major League Rookie of the Year',
       'Baseball America Major League Player of the Year','Home Run Derby Winner', 'All-Star MVP', 'AL All-Star',
       'NL All-Star',]


In [16]:
awardhist=training_dfs['awards'].copy()
awardhist=awardhist[['awardSeason','playerId','awardName']]
awardhist=awardhist.loc[awardhist.awardName.isin(awards),:]
awardhist=awardhist.rename(columns={'awardSeason':'seasonId'})
awardhist['seasonId']=awardhist['seasonId']+1
awardhist=pd.get_dummies(awardhist,'awardName')
awardhist=awardhist.groupby(['playerId','seasonId'],as_index=False).sum()
awardhist['playerId']=[str(a) for a in awardhist.playerId]

In [17]:
#features = [
#    "hits",
#    'doubles', 
#    'triples',
#    "strikeOuts",
#    "homeRuns",
#    "runsScored",
#    "stolenBases",
#    "strikes",
#    "flyOuts",
#    "groundOuts",
#    "errors",
#    "rbi",
#    "baseOnBalls",
#    "gamesPlayedPitching", 
#    'gamesStartedPitching',
#    'completeGamesPitching', 
#    'shutoutsPitching', 
#    'winsPitching',
#    'lossesPitching',
#    'runsPitching', 
#    'homeRunsPitching', 
#    'strikeOutsPitching',
#    'baseOnBallsPitching',
#    'inningsPitched',
#    'teamName',
#    'positionName',
#]


# game_date features

In [18]:
features = ['home',
    'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced',  'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances','battingOrder',
    'positionName'
]

In [19]:
gc.collect
del training

In [20]:
pids_test = players.playerId.loc[
    players.playerForTestSetAndFuturePreds.fillna(False)
].astype(str)
pids_test.shape

(1187,)

In [21]:
# Players in the test set. We'll filter our data for only this set of players
pids_test = players.playerId.loc[
    players.playerForTestSetAndFuturePreds.fillna(False)
].astype(str)

# Name of target columns
targets = ["target1", "target2", "target3", "target4"]

def make_playerBoxScores(dfs: dict, features):
    X = dfs['playerBoxScores'].copy()
    if 'playerId' in X.columns:
        X = dfs['playerBoxScores'].copy()
        X = X[['gameDate', 'playerId'] + features]
        X = X.rename(columns={'gameDate': 'date'})
        X['date'] = pd.PeriodIndex(X.date, freq='D')
        X=X[features+['date','playerId']]
        X['battingOrder']=X['battingOrder'].fillna(0)
        X.loc[X['battingOrder']==0,'battingOrder']=10009
        X['starting']=X['battingOrder']%100
        X['starting']=X['starting']==0
        X['order']=X['battingOrder']//100
        X=pd.get_dummies(X,columns=['order'])
        X.drop(['battingOrder'],axis=1,inplace=True)
        # Set dtypes
        X = X.astype({'playerId': str,'starting': int})
        X['positionName']=X['positionName'].fillna(value="X")
        # Create date index
        X=pd.get_dummies(data=X,columns=['positionName'])
        # Aggregate multiple games per day by summing
        X = X.groupby(['date', 'playerId'], as_index=False).mean()
    else:
        X[['playerId']]=np.nan
        X[['date']]=np.nan
    return X

# Set dtypes
#def make_playerTwitterFollowers(dfs:dict):
#    X=dfs['playerTwitterFollowers'].copy()
#    X=X[['date', 'playerId','numberOfFollowers']]
#    X['date'] = pd.PeriodIndex(X.date, freq='D')
##    # Aggregate multiple games per day by summing
#    X = X.groupby(['date', 'playerId'], as_index=False).sum()
#    return X

def make_targets(training_dfs: dict):
    Y = training_dfs['nextDayPlayerEngagement'].copy()
    # Set dtypes
    Y = Y.astype({name: np.float32 for name  in targets})
    Y = Y.astype({'playerId': str})
    # Match target dates to feature dates and create date index
    Y = Y.rename(columns={'engagementMetricsDate': 'date'})
    Y['date'] = pd.to_datetime(Y['date'])
    Y = Y.set_index('date').to_period('D')
    Y.index = Y.index - 1
    return Y.reset_index()


#def join_datasets(dfs):
#    dfs = [x.pivot(index='date', columns='playerId') for x in dfs]
#    df = pd.concat(dfs, axis=1).stack().reset_index('playerId')
#    return df

#def make_trainX(test_dfs: dict, features):
#    X = make_playerBoxScores(test_dfs, features)
#    X = X.merge(pids_test, how='right')
    #fourier_terms = CalendarFourier(freq='A', order=4)
    #deterministic = DeterministicProcess(
    #    index=X.index,
    #    order=0,
    #    seasonal=False,  # set to True for weekly seasonality
    #    additional_terms=[fourier_terms],
    #)
    #X = pd.concat([X, deterministic.in_sample()], axis=1)
 #   return X
#def make_trainy(training_dfs: dict,
#                      targets):

#    Y = make_targets(training_dfs)
    # Filter for players in test set
#    Y=Y.loc[Y.playerId.isin(pids_test), :]
    # Convert from long to wide format
    # Restore features and target
    # Fill missing values in features
#    return Y
    # Create train / validation splits
#X,Y=make_trainX(training_dfs,features),make_trainy(training_dfs,targets)

Before merging the tables, we create new features of "season part" since the atmosphere at the beginning of the season and that at the end of the season should be very different. We use 2021 season as test set, and 2019,2020 as training set.

In [22]:
def make_training_data(training_dfs: dict,
                       features,
                       targets,
                       fourier=4,
                       test_size=30):
    # Process dataframes
    X = make_playerBoxScores(training_dfs, features)
    standings=make_standings(training_dfs,standings_feature)
    games=make_games(training_dfs)
    rosters=make_rosters(training_dfs)
    #transactions=make_transactions(training_dfs)
    TwitterFollowers=make_playerTwitterFollowers(training_dfs)
    Y = make_targets(training_dfs)
    rosters['playerId']=[str(x) for x in rosters.playerId]
    players_1['playerId']=[str(x) for x in players_1.playerId]
    #transactions['playerId']=[str(x) for x in transactions.playerId]
    TwitterFollowers['playerId']=[str(x) for x in TwitterFollowers.playerId]
    # Merge for processing
    Y =Y.loc[Y.playerId.isin(pids_test), :]
    df = Y.merge(X,how='left',on=['date','playerId'])
    # Filter for players in test set
    df['seasonId']=df.date.dt.year
    df=df.loc[(df.seasonId!=2018),:]
    df=df.merge(awardhist,how='left',on=['seasonId','playerId'])
    df=df.merge(targ,how='left',on=['seasonId','playerId'])
    unique=pd.DataFrame({'date':df.date.unique()})
    unique['seasonId']=unique.date.dt.year
    unique=unique.merge(seasons,on='seasonId')
    unique['seasonPart'] = np.select(
        [
        unique['date'] < unique['preSeasonStartDate'], 
        unique['date'] <unique['regularSeasonStartDate'],
        unique['date'] <= unique['lastDate1stHalf'],
        unique['date'] < unique['firstDate2ndHalf'],
        unique['date'] <= unique['regularSeasonEndDate'],
        unique['date'] < unique['postSeasonStartDate'],
       unique['date'] <= unique['postSeasonEndDate'],
        unique['date'] >unique['postSeasonEndDate']
        ], 
        [
        'Offseason',
        'Preseason',
        'Reg Season 1st Half',
        'All-Star Break',
        'Reg Season 2nd Half',
        'Between Reg and Postseason',
        'Postseason',
        'Offseason'
        ], 
        default = np.nan
        )
    unique=unique[['date','seasonPart']]
    df=df.merge(unique,how='left',on='date')
    df=df.loc[df.seasonPart!='Offseason',:]
    #df=df.loc[df.seasonPart!='Preseason',:]
    #df=df.loc[df.seasonPart!='Postseason',:]
    #df=df.loc[df.seasonPart!='Offseason',:]
    df=df.drop('seasonId',axis=1)
    df=df.merge(rosters,how='left',on=['date','playerId'])
    #df=df.merge(transactions,how='left',on=['date','playerId'])
    df=df.merge(TwitterFollowers,how='left',on=['date','playerId'])
    df=df.merge(standings,how='left',on=['date','teamId'])
    df=df.merge(games,how='left',on=['date','teamId'])
    df=df.merge(players_1,how='left',on=['playerId'])
    #df['seasonId']=df.date.dt.year
    #df['age']=df['seasonId']-df['birthyear']
    #df.drop('seasonId',axis=1, inplace=True)
    df=pd.get_dummies(df,columns=['teamId'])
    # Convert from long to wide format
    #df = df.pivot(columns="playerId")
    # Restore features and targets
    df['seasonId']=df.date.dt.year
    df['age']=df['seasonId']-df['birthyear']
    train_df=df.loc[(df.seasonId!=2021),:]
    test_df=df.loc[(df.seasonId==2021),:]
    train_df=pd.get_dummies(train_df,columns=['seasonPart'])
    test_df=pd.get_dummies(test_df,columns=['seasonPart'])
    missing_cols = set( train_df.columns ) - set(test_df.columns )
    for c in missing_cols:
        test_df[c] = 0
    y_train = train_df[targets]
    y_valid = test_df[targets]
    # Fill missing values in features
    X_train =train_df.drop(targets+['date','playerId','birthyear','seasonId'],axis=1)
    X_train.fillna(0, inplace=True)
    X_valid =test_df.drop(targets+['date','playerId','birthyear','seasonId'],axis=1)
    X_valid.fillna(0, inplace=True)
    # Create temporal features
    # Create train / validation splits,:]
    #X_train,X_valid=X.loc[train_id,:],X.loc[test_id,:]
    #y_train,y_valid=Y.loc[train_id,:],Y.loc[test_id,:]
    return X_train, X_valid, y_train, y_valid

In [23]:
X_train, X_valid, y_train, y_valid=make_training_data(training_dfs,
                       features,
                       targets
                       )

In [24]:
X_train.columns

Index(['home', 'gamesPlayedBatting', 'flyOuts', 'groundOuts', 'runsScored',
       'doubles', 'triples', 'homeRuns', 'strikeOuts', 'baseOnBalls',
       ...
       'teamId_146.0', 'teamId_147.0', 'teamId_158.0', 'age',
       'seasonPart_All-Star Break', 'seasonPart_Between Reg and Postseason',
       'seasonPart_Postseason', 'seasonPart_Preseason',
       'seasonPart_Reg Season 1st Half', 'seasonPart_Reg Season 2nd Half'],
      dtype='object', length=215)

In [25]:
del training_dfs 

In [26]:

gc.collect()

18682

In [27]:
#X = training_dfs['playerBoxScores'].copy()
#X = X[['gameDate', 'playerId'] + features]
#object_cols = [col for col in features if X[col].dtype == "object"]
#num_cols = [col for col in features if col not in object_cols]
    # Set dtypes
#for name in num_cols:
#    X = X.astype({name: np.float32})
#X = X.astype({'playerId': str})
#X[object_cols]=X[object_cols].fillna(value="X")
    # Create date index
#X = X.rename(columns={'gameDate': 'date'})
#X['date'] = pd.PeriodIndex(X.date, freq='D')
    # Aggregate multiple games per day by summing
#X = X.groupby(['date', 'playerId'], as_index=False).sum()

In [28]:
#Y = training_dfs['nextDayPlayerEngagement'].copy()
# Set dtypes
#Y = Y.astype({name: np.float32 for name  in targets})
#Y = Y.astype({'playerId': str})
# Match target dates to feature dates and create date index
#Y = Y.rename(columns={'engagementMetricsDate': 'date'})
#Y['date'] = pd.to_datetime(Y['date'])
#Y = Y.set_index('date').to_period('D')
#Y.index = Y.index - 1
#Y=Y.reset_index()

In [29]:
#df = Y.merge(X,how='left',on=['date','playerId'])
# Filter for players in test set
#df = df.loc[df.playerId.isin(pids_test), :]

In [30]:
#object_cols = [col for col in features if df[col].dtype == "object"]


In [31]:
columns=X_train.columns
uint8=[col for col in columns if X_train[col].dtype == "uint8"]

In this cell we'll define the features we want to use from `playerBoxScores`. This frame contains player statistics for every game in the training period. We've chosen just a few from the many available. The `make_training_data` function also adjoins some features modeling annual seasonality, motivated by our data exploration in the next section.

# Data Exploration #

A *seasonal plot* can reveal seasonal effects in a time series. Let's look at seasonal plots for players in the top 10% of average engagement, one for yearly seasonality and one for weekly seasonality.

From these plots, digital player engagement appears to have a strong annual component, but little or no weekly component.

We can verify this with the *periodogram*. The periodogram illustrates the strength of the frequencies within a signal -- specifically, the variance of the sine / cosine Fourier component oscillating at that frequency.

Both of these visualizations indicate yearly (or annual) seasonality in the engagement time series, which motivated our decision to use yearly Fourier features when constructing our feature set.

# Model #

For this getting started notebook, we'll just use a simple feedforward network.

In [32]:
# Hyperparameters
#HIDDEN =100
#ACTIVATION = 'relu'  # could try elu, gelu, swish
#DROPOUT_RATE = 0.5
#LEARNING_RATE = 1e-3
#BATCH_SIZE = 100
#OUTPUTS = y_train.shape[-1]
#optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#tf.config.experimental_connect_to_cluster(tpu)
#tf.tpu.experimental.initialize_tpu_system(tpu)
#strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
#with strategy.scope():
#    model = keras.Sequential([
#        layers.Dense(HIDDEN, activation=ACTIVATION),
#        layers.BatchNormalization(),
#        layers.Dropout(DROPOUT_RATE),
#        layers.Dense(HIDDEN, activation=ACTIVATION),
#        layers.BatchNormalization(),
#        layers.Dropout(DROPOUT_RATE),
#        layers.Dense(OUTPUTS),
#    ])
#    model.compile(optimizer=optimizer, loss='mae', metrics=['mae'],steps_per_execution=64)
#early_stopping = keras.callbacks.EarlyStopping(patience=10)
#x_trial=np.asarray(X_train).astype('float32')
#x_vatrial=np.asarray(X_valid).astype('float32')
#history = model.fit(
#    x_trial, y_train,
#    validation_data=(x_vatrial, y_valid),
#    batch_size=BATCH_SIZE,
#    epochs=1000,
#    callbacks=[early_stopping],
#)
#model.save('my_model.h5')
    

In [33]:
gc.collect()

40

In [34]:
#history_df = pd.DataFrame(history.history)
#history_df.loc[:, ['loss', 'val_loss']].plot()

In [35]:
gc.collect

del y_train
del X_valid,y_valid,X_train

# Create Submission #

The next cell illustrates how you to create a submission for this competition. As this is a code competition that relies on a time series module, submissions must follow the requirements described on the [Evaluation Page](https://www.kaggle.com/c/mlb-player-digital-engagement-forecasting/overview/evaluation) and [Data Page](https://www.kaggle.com/c/mlb-player-digital-engagement-forecasting/data).

In [36]:
import joblib
model_1=joblib.load("../input/mlb-with-lag/Class_1.pkl")
model_2=joblib.load("../input/mlb-with-lag/Class_2.pkl")
model_3=joblib.load("../input/mlb-with-lag/Class_3.pkl")
model_4=joblib.load("../input/mlb-with-lag/Class_4.pkl")

In [37]:
def make_predictions(targets,test_dfs,features):
    X = make_playerBoxScores(test_dfs, features)
    standings=make_standings(test_dfs,standings_feature)
    rosters=make_rosters(test_dfs)
    games=make_games(test_dfs)
    TwitterFollowers=make_playerTwitterFollowers(test_dfs)
    rosters['playerId']=[str(x) for x in rosters.playerId]
    TwitterFollowers['playerId']=[str(x) for x in TwitterFollowers.playerId]
    players_1['playerId']=[str(x) for x in players_1.playerId]
    X = X.merge(pids_test, how='right')
    
    X['date'] = X.date.fillna(method='ffill').fillna(method='bfill')
        # Merge for processing
        # Filter for players in test set
    X['seasonId']=X.date.dt.year
    X=X.merge(awardhist,how='left',on=['seasonId','playerId'])
    X=X.merge(targ,how='left',on=['seasonId','playerId'])
    X=X.merge(seasons,how='left',on='seasonId')
    X['seasonPart'] = np.select(
            [
            X['date'] < X['preSeasonStartDate'], 
            X['date'] <X['regularSeasonStartDate'],
            X['date'] <= X['lastDate1stHalf'],
            X['date'] <X['firstDate2ndHalf'],
            X['date'] <=X['regularSeasonEndDate'],
            X['date'] < X['postSeasonStartDate'],
           X['date'] <= X['postSeasonEndDate'],
            X['date'] >X['postSeasonEndDate']
            ], 
            [
            'Offseason',
            'Preseason',
            'Reg Season 1st Half',
            'All-Star Break',
            'Reg Season 2nd Half',
            'Between Reg and Postseason',
            'Postseason',
            'Offseason'
            ], 
            default = np.nan
            )
    X=pd.get_dummies(X,columns=['seasonPart'])
    X=X.drop('seasonId',axis=1)
    X=X.merge(rosters,how='left',on=['date','playerId'])
    X=X.merge(TwitterFollowers,how='left',on=['date','playerId'])
    X=X.merge(standings,how='left',on=['date','teamId'])
    X=X.merge(games,how='left',on=['date','teamId'])
    X=X.merge(players_1,how='left',on=['playerId'])  
    X['seasonId']=X.date.dt.year
    X['age']=X['seasonId']-X['birthyear']
    X.drop(['seasonId','birthyear'],axis=1, inplace=True)
    missing_cols = set( columns ) - set(X.columns )
        # Add a missing column in test set with default value equal to 0
    for c in missing_cols:
        X[c] = 0
    X.fillna(0, inplace=True)
    A=X.copy()
    A=A[columns]
    A=A.astype('float32')
    A = A.astype({name: np.int for name  in uint8})
    A=A.astype({name:np.int for name in ["wildeliminated","eliminated"]})
    targets1 = model_1.predict(A)
    targets2= model_2.predict(A)
    targets3=model_3.predict(A)
    targets4=model_4.predict(A)
    y_pred = pd.DataFrame({"target1":targets1,"target2":targets2,"target3":targets3,"target4":targets4})
    y_pred[targets] = y_pred[targets].clip(0, 100)
    y_pred['date']=X['date']
    y_pred['playerId'] =X.playerId
    y_pred['date']=y_pred.date.fillna(method='ffill').fillna(method='bfill')
    y_pred['date']=[(x+1).strftime('%Y%m%d') for x in y_pred['date']]
    y_pred['date_playerId']=[b['date']+'_'+str(b['playerId']) for a,b in y_pred.iterrows()]
    y_pred=y_pred.drop(['date','playerId'],axis=1)
    y_pred = y_pred[['date_playerId'] + targets]
#        (date + 1).strftime('%Y%m%d') + '_' + str(playerId)
#        for date, playerId in y_pred.index
#    
#    y_pred.reset_index('playerId', drop=True, inplace=True)
#    y_pred = y_pred[['date_playerId'] + targets]  # reorder
#    y_pred.index = pd.Int64Index(
#        [int(date.strftime('%Y%m%d')) for date in y_pred.index], name='date')
    return y_pred

In [38]:
%%time
import mlb

env = mlb.make_env()
iter_test = env.iter_test()

for (test_df, sample_prediction_df) in iter_test:
#    # Unpack features from test_df
    test_dfs = unpack_data(test_df, [  # targets
    'playerBoxScores',  # features
    # Other dataframes available for features:
    'games',
    'rosters',
    'teamBoxScores',
    'transactions',
    'standings',
    # 'awards',
    # 'events',
    'playerTwitterFollowers',
    'teamTwitterFollowers',
])
    # Create predictions
    y_pred = make_predictions(
        test_dfs=test_dfs,
        features=features,
        targets=targets,
    )
    submission = (
       sample_prediction_df
        [['date_playerId']]
        .reset_index()  
        .merge(y_pred, how='left', on='date_playerId')
        .set_index('date')  
    )

    # Submit predictions
    env.predict(submission)  # constructs submissions.csv


This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
CPU times: user 11.6 s, sys: 41 ms, total: 11.6 s
Wall time: 10.7 s


In [39]:
submission

,date_playerId,target1,target2,target3,target4
date,,,,,
20210430,20210501_488726,2.246578,6.239217,0.060968,2.489672
20210430,20210501_605218,0.002695,0.423651,0.006261,0.787937
20210430,20210501_621563,0.224530,3.262513,0.013906,0.977115
20210430,20210501_670084,0.004608,1.355075,0.025096,0.702753
20210430,20210501_670970,0.001378,0.437157,0.122985,0.168838
...,...,...,...,...,...
20210430,20210501_596049,0.000000,0.133644,0.002524,0.028806
20210430,20210501_642851,0.006739,0.209232,0.005120,0.186146
20210430,20210501_596071,0.000005,0.237598,0.000104,0.114882


To complete a submission for this competition, we'll need to commit this notebook and submit the resulting submission file it creates,`submissions.csv`. From the notebook editor:
- make sure *Internet* is turned off in **Settings**,
- click the **Save Version** button to the upper right,
- make sure *Save and Run All (Commit)* is selected, and
- click **Save**.

Once the commit completes (should only be three or four minutes):
- select from the menubar **File -> Version History**,
- select from the ellipsis menu of the latest version **... -> Submit to Competition**, and
- click **Submit**

Kaggle will rerun the notebook on the public test set and display the score on the public leaderboard. [**My Submissions**](https://www.kaggle.com/c/mlb-player-digital-engagement-forecasting/submissions) will show the status of the submission: currently running, succeeded, or failed.

The submission requirements for code competitions can be somewhat exacting. If you're having trouble, checkout our [**Code Competitions - Errors & Debugging Tips**](https://www.kaggle.com/docs/competitions#notebooks-only-FAQ).

# Explainable AI and Hyperparameter Tuning with Vertex #

[This complementary notebook](https://www.kaggle.com/ryanholbrook/vertex-ai-with-mlb-player-digital-engagement) demonstrates how to run this notebook in Vertex AI Notebooks. You'll learn how to use **Explainable AI (XAI)** to refine your features and how to tune your model with **Vertex Vizier**.